In [1]:
import tensorflow as tf

2022-12-18 20:34:34.107497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 20:34:34.252574: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-18 20:34:34.257677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-18 20:34:34.257696: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
import os
import numpy as np
import random

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
MODEL_NAME = 'model4'

PREPROCESSING_ARGS = {
    'downsampling_rate' : 16000,
    'frame_length_in_s' : 0.032,
    'frame_step_in_s' : 0.032,
    'num_mel_bins': 10,  
    'lower_frequency' : 20,  
    'upper_frequency' : 8000,  
    'num_coefficients' : 10
}

FILTERS = 128  # filters used in the convolution

alpha = 0.5

LABELS = ['stop', 'go']  # ATTENTION, MUST BE SAME ORDER

TRAINING_ARGS = {
    'batch_size': 20,
    'initial_learning_rate': 0.01,
    'end_learning_rate': 1.e-5,
    'epochs': 10
}

final_sparsity = 0.7

In [4]:
from preprocessing import get_mfccs
from functools import partial

# use only go and stop words
train_ds = tf.data.Dataset.list_files(['msc-train/go*', 'msc-train/stop*'])
val_ds = tf.data.Dataset.list_files(['msc-val/go*', 'msc-val/stop*'])
test_ds = tf.data.Dataset.list_files(['msc-test/go*', 'msc-test/stop*'])

get_frozen_mfccs = partial(get_mfccs, **PREPROCESSING_ARGS)

# save the shape of the mfccs
for mfccs, label in train_ds.map(get_frozen_mfccs).take(1):
    SHAPE = mfccs.shape

2022-12-18 20:34:35.682933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-18 20:34:35.682958: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-18 20:34:35.682976: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-919441e0-aa3f-4bac-a9dc-75daa4d8d934): /proc/driver/nvidia/version does not exist
2022-12-18 20:34:35.683224: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 20:34:36.394625: W tensorflow_io/core/kernels/audio_v

In [5]:
def preprocess(filename):
    signal, label = get_frozen_mfccs(filename)

    signal.set_shape(SHAPE)
    signal = tf.expand_dims(signal, -1)
    # signal = tf.image.resize(signal, [32, 32]) # not needed since dimension is smaller than spectogram and it will add more latency
                                                 # in the inference preprocessing
    label_id = tf.argmax(label == LABELS)  # change labels from text to a number

    return signal, label_id

batch_size = TRAINING_ARGS['batch_size']
epochs = TRAINING_ARGS['epochs']

# create batches
train_ds = train_ds.map(preprocess).batch(batch_size).cache()
val_ds = val_ds.map(preprocess).batch(batch_size)
test_ds = test_ds.map(preprocess).batch(batch_size)

2022-12-18 20:34:37.481335: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-18 20:34:37.482877: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-18 20:34:37.483044: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-18 20:34:37.813319: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-18 20:34:37.814689: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-18 20:34:37.814866: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-18 20:34:38.138048: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed 

In [6]:
for example_batch, example_labels in train_ds.take(1):
    pass

2022-12-18 20:34:38.571829: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [7]:
# define model

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(FILTERS*alpha), kernel_size=[3, 3], strides=[2, 2],  # always start with a standard
    use_bias=False, padding='valid'),                                        # convolution, but increasing the
    tf.keras.layers.BatchNormalization(),                                    # number of filter to compensate 
    tf.keras.layers.ReLU(),                                                  # for the loss of parameters
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
    use_bias=False, padding='same'),
    tf.keras.layers.Conv2D(filters=int(FILTERS*alpha), kernel_size=[1, 1], strides=[1, 1],
    use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],      # don't need the number of filter, because is 1 by
    use_bias=False, padding='same'),                                         # definition, this is a convolution over the H and W
    tf.keras.layers.Conv2D(filters=int(FILTERS*alpha), kernel_size=[1, 1], strides=[1, 1],  # point-wise convolution (kernel = [1,1])
    use_bias=False),                                                         # to do the convolution over the channels C
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])

In [8]:
# define model for pruning

import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude  # rename the function

begin_step = int(len(train_ds) * epochs * 0.2)  # number of batches * number of epochs is the number of steps
                                                # we start at 20%
end_step = int(len(train_ds) * epochs)

pruning_params = {
    'pruning_schedule' : tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.20,
        final_sparsity=final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )  # scheduler
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

In [9]:
# train the model

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds) * epochs,
)

optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]

callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model_for_pruning.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)

Epoch 1/10
80/80 [==============================] - 8s 62ms/step - loss: 0.3032 - sparse_categorical_accuracy: 0.8756 - val_loss: 0.3727 - val_sparse_categorical_accuracy: 0.8650
Epoch 2/10
80/80 [==============================] - 2s 24ms/step - loss: 0.1475 - sparse_categorical_accuracy: 0.9481 - val_loss: 0.1632 - val_sparse_categorical_accuracy: 0.9550
Epoch 3/10
80/80 [==============================] - 2s 23ms/step - loss: 0.0942 - sparse_categorical_accuracy: 0.9675 - val_loss: 0.2615 - val_sparse_categorical_accuracy: 0.9000
Epoch 4/10
80/80 [==============================] - 2s 22ms/step - loss: 0.0690 - sparse_categorical_accuracy: 0.9769 - val_loss: 0.2677 - val_sparse_categorical_accuracy: 0.9150
Epoch 5/10
80/80 [==============================] - 2s 22ms/step - loss: 0.0566 - sparse_categorical_accuracy: 0.9794 - val_loss: 0.2556 - val_sparse_categorical_accuracy: 0.9450
Epoch 6/10
80/80 [==============================] - 2s 23ms/step - loss: 0.0460 - sparse_categorical_accu

In [10]:
# test model

test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)

10/10 [==============================] - 1s 38ms/step - loss: 0.0410 - sparse_categorical_accuracy: 0.9850


In [11]:
# store and print results

training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

Training Loss: 0.0324
Training Accuracy: 99.25%

Validation Loss: 0.0774
Validation Accuracy: 95.50%

Test Loss: 0.0410
Test Accuracy: 98.50%


In [12]:
# save the model

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)  # convert the model

# saved_model_dir = f'./saved_models/{timestamp}'
saved_model_dir = f'./saved_models/{MODEL_NAME}'
if not os.path.exists(saved_model_dir):
    os.makedirs(saved_model_dir)
model_for_export.save(saved_model_dir)

INFO:tensorflow:Assets written to: ./saved_models/model4/assets
INFO:tensorflow:Assets written to: ./saved_models/model4/assets


In [13]:
# convert model to tflite

converter = tf.lite.TFLiteConverter.from_saved_model(f'./saved_models/{MODEL_NAME}')
tflite_model = converter.convert()

2022-12-18 20:35:06.472832: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-18 20:35:06.472871: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-18 20:35:06.473457: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./saved_models/model4
2022-12-18 20:35:06.475777: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-18 20:35:06.475802: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./saved_models/model4
2022-12-18 20:35:06.479778: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-18 20:35:06.481129: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-18 20:35:06.514866: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./saved_models

In [14]:
tflite_models_dir = './tflite_models'

if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)

In [15]:
tflite_model_name = os.path.join(tflite_models_dir, f'{MODEL_NAME}.tflite')

with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)

In [16]:
# compress tflite model

import zipfile

with zipfile.ZipFile(f'{tflite_model_name}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(tflite_model_name)

In [17]:
original_model_size = os.path.getsize(f'./saved_models/{MODEL_NAME}/saved_model.pb') / 1024
pruned_tflite_size = os.path.getsize(tflite_model_name) / 1024
pruned_zip_size = os.path.getsize(f'{tflite_model_name}.zip') / 1024

print(f'Original Size (pruned model): {original_model_size:.2f} KB')
print(f'Original TFLite Size (pruned model): {pruned_tflite_size:.2f} KB')
print(f'ZIP TFLite Size (pruned model): {pruned_zip_size:.2f} KB')

Original Size (pruned model): 223.33 KB
Original TFLite Size (pruned model): 44.02 KB
ZIP TFLite Size (pruned model): 20.64 KB


In [18]:
import pandas as pd

output_dict = {
    # 'timestamp': timestamp,
    **PREPROCESSING_ARGS,
    'filters' : FILTERS,
    'final_sparsity' : final_sparsity,
    'original_size' : original_model_size,
    'tflite_size' : pruned_tflite_size,
    'zip_size' : pruned_zip_size,
    **TRAINING_ARGS,
    'tflite_preprocessing_latency' : -1,
    'tflite_model_latency' : -1,
    'tflite_total_latency' : -1,
    'original_test_accuracy': test_accuracy,
    'tflite_test_accuracy' : -1
}

df = pd.DataFrame([output_dict])

output_path='./training_results.csv'
# df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=919441e0-aa3f-4bac-a9dc-75daa4d8d934' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>